In [9]:
import numpy as np
import pandas as pd
import requests

pd.options.display.max_colwidth = 1000

In [10]:
hostname = "http://lookup-service-prod.mlb.com"
path = "/json/named.transaction_all.bam?sport_code='mlb'&start_date={0}&end_date={1}".format("20180101","20181231")

In [11]:
r = requests.get('{0}{1}'.format(hostname, path))
j = r.json()

In [12]:
df = pd.DataFrame(j['transaction_all']['queryResults']['row'])

In [16]:
placed = df[(df.note.str.contains("placed")) & (df.note.str.contains("disabled list"))]

,trans_date_cd,from_team_id,orig_asset,final_asset_type,player,resolution_cd,final_asset,name_display_first_last,type_cd,name_sort,...,type,name_display_last_first,transaction_id,trans_date,effective_date,player_id,orig_asset_type,from_team,team_id,note
732,D,,Player,,Grant Dayton,FIN,,Grant Dayton,SC,"DAYTON, GRANT",...,Status Change,"Dayton, Grant",341769,2018-02-14T00:00:00,2018-02-14T00:00:00,594795,PL,,144,Atlanta Braves placed LHP Grant Dayton on the 60-day disabled list.
742,D,,Player,,Zack Britton,FIN,,Zack Britton,SC,"BRITTON, ZACK",...,Status Change,"Britton, Zack",341838,2018-02-15T00:00:00,2018-02-15T00:00:00,502154,PL,,110,Baltimore Orioles placed LHP Zach Britton on the 60-day disabled list. Right Achilles surgery.
760,D,,Player,,Drew Smyly,FIN,,Drew Smyly,SC,"SMYLY, DREW",...,Status Change,"Smyly, Drew",341914,2018-02-16T00:00:00,2018-02-16T00:00:00,592767,PL,,112,Chicago Cubs placed LHP Drew Smyly on the 60-day disabled list. Recovery from Tommy John surgery.
769,D,,Player,,Michael Pineda,FIN,,Michael Pineda,SC,"PINEDA, MICHAEL",...,Status Change,"Pineda, Michael",341974,2018-02-17T00:00:00,2018-02-17T00:00:00,501381,PL,,142,Minnesota Twins placed RHP Michael Pineda on the 60-day disabled list. Right UCL injury.
784,D,,Player,,T.J. Rivera,FIN,,T.J. Rivera,SC,"RIVERA, T.J.",...,Status Change,"Rivera, T.J.",341987,2018-02-18T00:00:00,2018-02-18T00:00:00,608061,PL,,121,New York Mets placed 3B T.J. Rivera on the 60-day disabled list. Tommy John surgery.


In [44]:
row = dict()
df3 = pd.DataFrame(columns = ['playerid','injury_start_date','injury_end_date'])

for k,v in placed.iterrows():
    row['playerid'] = v.player_id
    row['injury_start_date'] = v.trans_date
    
    # resolve this players time injured
    activated = df[(df.player_id == v.player_id) & 
                   (df.trans_date > v.trans_date) & 
                   ((df.note.str.contains("activated")) & ((df.note.str.contains("disabled list")) | (df.note.str.contains("injured list"))))]
    optioned = df[(df.player_id == v.player_id) & 
                  (df.trans_date > v.trans_date) & 
                  (df.type.isin(['Optioned','Released','Signed as Free Agent',
                                 'Declared Free Agency','Outrighted','Returned',
                                 'Claimed Off Waivers','Retired']))]
    combined = activated.append(other=optioned)
    combined = combined.sort_values(by=['trans_date'])
    
    if len(combined) > 0:
        row['injury_end_date'] = combined.iloc[0].trans_date
    else:
        activated = df[(df.player_id == v.player_id) & 
                   (df.trans_date > v.trans_date) & 
                   (df.note.str.contains("activated"))]
        combined = activated.sort_values(by=['trans_date'])
        if len(combined) > 0:
            row['injury_end_date'] = combined.iloc[0].trans_date
        else:
            activated = df[(df.player_id == v.player_id) & 
                   (df.trans_date > v.trans_date) & 
                   (df.note.str.contains("roster status changed by"))]
            combined = activated.sort_values(by=['trans_date'])
            if len(combined) > 0:
                row['injury_end_date'] = combined.iloc[0].trans_date
            
    tmp = pd.Series(row)
    df3 = df3.append(other=tmp, ignore_index=True)
    row = dict()
    
df3.head()

,playerid,injury_start_date,injury_end_date
0,594795,2018-02-14T00:00:00,2018-10-31T00:00:00
1,502154,2018-02-15T00:00:00,2018-10-29T00:00:00
2,592767,2018-02-16T00:00:00,NaN
3,501381,2018-02-17T00:00:00,2018-10-29T00:00:00
4,608061,2018-02-18T00:00:00,2018-11-02T00:00:00


In [45]:
df3[df3.injury_end_date.isnull()]

,playerid,injury_start_date,injury_end_date
2,592767,2018-02-16T00:00:00,NaN
16,605194,2018-03-19T00:00:00,NaN
701,640455,2018-08-26T00:00:00,NaN
736,453214,2018-10-06T00:00:00,NaN


In [27]:
df3.to_csv("./2018_injury_mlb_api.csv")

In [49]:
df[df.player_id == '453214'].loc[:,['type','trans_date','resolution_date','note']]

,type,trans_date,resolution_date,note
2979,Status Change,2018-03-26T00:00:00,2018-03-26T00:00:00,Boston Red Sox placed RHP Steven Wright on the 10-day disabled list. Recovery from cartilage restoration procedure.
3915,Assigned,2018-04-20T00:00:00,2018-04-20T00:00:00,Boston Red Sox sent RHP Steven Wright on a rehab assignment to Pawtucket Red Sox.
4207,Status Change,2018-04-28T00:00:00,2018-04-28T00:00:00,Boston Red Sox placed RHP Steven Wright on the restricted list.
4769,Status Change,2018-05-14T00:00:00,2018-05-14T00:00:00,Boston Red Sox activated RHP Steven Wright from the restricted list.
7363,Status Change,2018-06-26T00:00:00,2018-06-23T00:00:00,"Boston Red Sox placed RHP Steven Wright on the 10-day disabled list retroactive to June 23, 2018. Left knee inflammation."
10569,Assigned,2018-08-28T00:00:00,2018-08-28T00:00:00,Boston Red Sox sent RHP Steven Wright on a rehab assignment to Pawtucket Red Sox.
10764,Status Change,2018-09-01T00:00:00,2018-09-01T00:00:00,Boston Red Sox activated RHP Steven Wright from the 10-day disabled list.
11411,Status Change,2018-10-06T00:00:00,,Boston Red Sox placed RHP Steven Wright on the 10-day disabled list. Left knee inflammation.
